In [4]:
!pip install psycopg2-binary

In [16]:
import psycopg2
import pandas as pd
import pdb
import time
from CategoryPredictor import CategoryPredictor

In [5]:
import psycopg2
import pandas as pd
import os

DB_HOST=os.environ.get('DB_HOST')
DB_NAME=os.environ.get('DB_NAME')
DB_USER=os.environ.get('DB_USER')
DB_PASSWORD=os.environ.get('DB_PASSWORD')

conn = psycopg2.connect(dbname=DB_NAME, user=DB_USER, 
                        password=DB_USER, host=DB_HOST)
cursor = conn.cursor()

In [7]:
max_id = pd.read_sql('SELECT max(id) FROM tag_project', conn)['max'][0]
max_id

/var/folders/64/nbt92b3d4_n2r41g3gx6lm340000gn/T/ipykernel_5435/2925573434.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  max_id = pd.read_sql('SELECT max(id) FROM tag_project', conn)['max'][0]


9

subsidies

subsidy_project

In [ ]:
f'''SELECT {self.categorys}.title AS subsidy, projects.title, {self.categorys}.id,
                    {self.join_table}.accepted FROM {self.join_table}
                    LEFT JOIN subsidies ON ({self.join_table}.subsidies_id = {self.categorys}.id)
                    LEFT JOIN projects ON ({self.join_table}.project_id = projects.id);'''

In [7]:
import psycopg2
import pandas as pd
import pdb
import time
from CategoryPredictor import CategoryPredictor

class PredictController:
    def connect_db(self):
        self.conn = psycopg2.connect(dbname='ideas', user='ideas', 
                        password='ideas2022', host='collabro.ru')
        self.cursor = self.conn.cursor()

    def get_data_db(self):
        q = f'''SELECT {self.categorys}.{self.category_col} AS subsidy, projects.title, {self.categorys}.id,
                    {self.join_table}.accepted FROM {self.join_table}
                    LEFT JOIN {self.categorys} ON ({self.join_table}.{self.category_id} = {self.categorys}.id)
                    LEFT JOIN projects ON ({self.join_table}.project_id = projects.id);'''
        if self.categorys == 'tag':
            print(q)
        df = pd.read_sql(q, self.conn)
        self.len_sub_project = len(df)

        df_train = df[df['accepted'] == True]

        df_test = pd.read_sql('SELECT * FROM projects', self.conn)
        return df_train, df_test
        
    def write_predictions(self, row):
        q = f'''SELECT * FROM {self.categorys}'''
        df_sub = pd.read_sql(q, self.conn)
        project_id = row['id']
        predictions = row['predictions']
        dsubsidy_project = pd.read_sql(f'SELECT * FROM {self.join_table}', self.conn)
        for subsidy in predictions:
            subsidy_id = df_sub[df_sub[self.category_col] == subsidy]['id'].iloc[0]
            prob = predictions[subsidy]
            print(prob, self.len_sub_project)
            if len(dsubsidy_project[(dsubsidy_project['project_id'] == project_id) & (dsubsidy_project[self.category_id] == subsidy_id)]) == 0:
                
                self.values.append((int(self.len_sub_project + 1), False, float(prob), int(project_id), int(subsidy_id)))
                self.len_sub_project += 1
            
            self.conn.commit()

    def fit_predict(self, categorys='subsidies', join_table='subsidy_project', category_col='title', category_id='subsidies_id'):
        self.categorys = categorys
        self.join_table=join_table
        self.category_col=category_col
        self.category_id=category_id
        df_train, df_test = self.get_data_db()
        Predictor = CategoryPredictor(label='title',    
                              category_columns=['subsidy'], df=df_train)
        Predictor.fit_all()
        df_predicted = Predictor.predict(labels=df_test, category='subsidy')
        self.values = []
        df_predicted.apply(self.write_predictions, axis=1)

        if self.values != []:
            q = f'''INSERT INTO {self.join_table} (id, accepted, probability, project_id, {self.category_id}) VALUES ''' + '{}'
            records_list_template = ','.join(['%s'] * len(self.values))
            insert_query = q.format(records_list_template)
            cursor = self.conn.cursor()
            cursor.execute(insert_query, self.values)
            self.conn.commit()
        
        return df_predicted

    def server(self):
        while True:
            self.connect_db()
            self.fit_predict(categorys='subsidies', join_table='subsidy_project', category_col='title', category_id='subsidies_id')
            time.sleep(1)
            self.fit_predict(categorys='tag', join_table='tag_project', category_col='tag_name', category_id='tag_id')
            time.sleep(1)
            self.conn.close()
            time.sleep(10)

Controller = PredictController()
df_pred = Controller.server()

/var/folders/64/nbt92b3d4_n2r41g3gx6lm340000gn/T/ipykernel_77782/2977689797.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(q, self.conn)
/var/folders/64/nbt92b3d4_n2r41g3gx6lm340000gn/T/ipykernel_77782/2977689797.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_test = pd.read_sql('SELECT * FROM projects', self.conn)
/Users/lebedevmatvey/Documents/Nlp_fastWin/CategoryPredictor.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

score 3 error
score 3 error
score 3 error
score 3 error
score 3 error
score 3 error
score 3 error
score 3 error
score 3 error
score 3 error
score 3 error
score 3 error
score 3 error
score 3 error
score 3 error
score 3 error
score 3 error
score 3 error
score 3 error
score 3 error
score 3 error
score 3 error
score 3 error
score 3 error
score 3 error
score 3 error
score 3 error
score 3 error
score 3 error
score 3 error
score 3 error
score 3 error
score 3 error
score 3 error
score 3 error
score 3 error
score 3 error


/var/folders/64/nbt92b3d4_n2r41g3gx6lm340000gn/T/ipykernel_77782/2977689797.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sub = pd.read_sql(q, self.conn)
/var/folders/64/nbt92b3d4_n2r41g3gx6lm340000gn/T/ipykernel_77782/2977689797.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dsubsidy_project = pd.read_sql(f'SELECT * FROM {self.join_table}', self.conn)


8.743723436350955 15
0 15
0.0 15


/var/folders/64/nbt92b3d4_n2r41g3gx6lm340000gn/T/ipykernel_77782/2977689797.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sub = pd.read_sql(q, self.conn)
/var/folders/64/nbt92b3d4_n2r41g3gx6lm340000gn/T/ipykernel_77782/2977689797.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dsubsidy_project = pd.read_sql(f'SELECT * FROM {self.join_table}', self.conn)


...

/var/folders/64/nbt92b3d4_n2r41g3gx6lm340000gn/T/ipykernel_77782/2977689797.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sub = pd.read_sql(q, self.conn)
/var/folders/64/nbt92b3d4_n2r41g3gx6lm340000gn/T/ipykernel_77782/2977689797.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dsubsidy_project = pd.read_sql(f'SELECT * FROM {self.join_table}', self.conn)


0.5652270793541772 15
0.0 15


/var/folders/64/nbt92b3d4_n2r41g3gx6lm340000gn/T/ipykernel_77782/2977689797.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sub = pd.read_sql(q, self.conn)
/var/folders/64/nbt92b3d4_n2r41g3gx6lm340000gn/T/ipykernel_77782/2977689797.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dsubsidy_project = pd.read_sql(f'SELECT * FROM {self.join_table}', self.conn)


0.19890262674252532 15
0.0947307954224878 15
0.0 15


KeyboardInterrupt: 

In [4]:
Controller.conn

<connection object at 0x13c716e30; dsn: 'user=ideas password=xxx dbname=ideas host=collabro.ru', closed: 0>

In [8]:
Controller = PredictController()
df_pred = Controller.fit_predict()

/var/folders/64/nbt92b3d4_n2r41g3gx6lm340000gn/T/ipykernel_74450/3455301127.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(q, self.conn)
/var/folders/64/nbt92b3d4_n2r41g3gx6lm340000gn/T/ipykernel_74450/3455301127.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_test = pd.read_sql('SELECT * FROM projects', self.conn)
/Users/lebedevmatvey/Documents/Nlp_fastWin/CategoryPredictor.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

8.743723436350955 15
2.0660720616806016 15
0.0 15


/var/folders/64/nbt92b3d4_n2r41g3gx6lm340000gn/T/ipykernel_74450/3455301127.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sub = pd.read_sql(q, self.conn)
/var/folders/64/nbt92b3d4_n2r41g3gx6lm340000gn/T/ipykernel_74450/3455301127.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dsubsidy_project = pd.read_sql(f'SELECT * FROM {self.join_table}', self.conn)


45.15295172031554 15
0.30631004518348903 15
0.062106374599617285 15


/var/folders/64/nbt92b3d4_n2r41g3gx6lm340000gn/T/ipykernel_74450/3455301127.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sub = pd.read_sql(q, self.conn)
/var/folders/64/nbt92b3d4_n2r41g3gx6lm340000gn/T/ipykernel_74450/3455301127.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dsubsidy_project = pd.read_sql(f'SELECT * FROM {self.join_table}', self.conn)


4.200796450364289 15
0.0 15
0.0 15


/var/folders/64/nbt92b3d4_n2r41g3gx6lm340000gn/T/ipykernel_74450/3455301127.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sub = pd.read_sql(q, self.conn)
/var/folders/64/nbt92b3d4_n2r41g3gx6lm340000gn/T/ipykernel_74450/3455301127.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dsubsidy_project = pd.read_sql(f'SELECT * FROM {self.join_table}', self.conn)


21.38593835685541 15
0.5652270793541772 15
0.035489356914067016 15


/var/folders/64/nbt92b3d4_n2r41g3gx6lm340000gn/T/ipykernel_74450/3455301127.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sub = pd.read_sql(q, self.conn)
/var/folders/64/nbt92b3d4_n2r41g3gx6lm340000gn/T/ipykernel_74450/3455301127.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dsubsidy_project = pd.read_sql(f'SELECT * FROM {self.join_table}', self.conn)


23.82323871512987 15
0.19890262674252532 15
0.0947307954224878 15


In [9]:
Controller = PredictController()
Controller.server()

/var/folders/64/nbt92b3d4_n2r41g3gx6lm340000gn/T/ipykernel_74450/3455301127.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(q, self.conn)
/var/folders/64/nbt92b3d4_n2r41g3gx6lm340000gn/T/ipykernel_74450/3455301127.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_test = pd.read_sql('SELECT * FROM projects', self.conn)
/Users/lebedevmatvey/Documents/Nlp_fastWin/CategoryPredictor.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

8.743723436350955 15
2.0660720616806016 15
0.0 15


/var/folders/64/nbt92b3d4_n2r41g3gx6lm340000gn/T/ipykernel_74450/3455301127.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dsubsidy_project = pd.read_sql(f'SELECT * FROM {self.join_table}', self.conn)
/var/folders/64/nbt92b3d4_n2r41g3gx6lm340000gn/T/ipykernel_74450/3455301127.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sub = pd.read_sql(q, self.conn)


45.15295172031554 15
0.30631004518348903 15
0.062106374599617285 15


/var/folders/64/nbt92b3d4_n2r41g3gx6lm340000gn/T/ipykernel_74450/3455301127.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dsubsidy_project = pd.read_sql(f'SELECT * FROM {self.join_table}', self.conn)
/var/folders/64/nbt92b3d4_n2r41g3gx6lm340000gn/T/ipykernel_74450/3455301127.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sub = pd.read_sql(q, self.conn)


...

4.981049792006358 15
2.0485000891963243 15


/var/folders/64/nbt92b3d4_n2r41g3gx6lm340000gn/T/ipykernel_74450/3455301127.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dsubsidy_project = pd.read_sql(f'SELECT * FROM {self.join_table}', self.conn)


1.5836595876084634 15
0.3353580036515372 15


/var/folders/64/nbt92b3d4_n2r41g3gx6lm340000gn/T/ipykernel_74450/3455301127.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(q, self.conn)
/var/folders/64/nbt92b3d4_n2r41g3gx6lm340000gn/T/ipykernel_74450/3455301127.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_test = pd.read_sql('SELECT * FROM projects', self.conn)
/Users/lebedevmatvey/Documents/Nlp_fastWin/CategoryPredictor.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

ValueError: math domain error

In [86]:
dsubsidy_project = pd.read_sql('SELECT * FROM subsidy_project', Controller.conn)
len(dsubsidy_project[(dsubsidy_project['project_id'] == 1) & (dsubsidy_project['subsidies_id'] == 1)])

/var/folders/64/nbt92b3d4_n2r41g3gx6lm340000gn/T/ipykernel_33082/2081191725.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dsubsidy_project = pd.read_sql('SELECT * FROM subsidy_project', Controller.conn)


2

In [ ]:
Controller = PredictController()
df_pred = Controller.fit_predict()

In [176]:
df_pred

,id,description,is_innovation,title,predictions
0,1,Экологический мониторинг атмосферного воздуха,False,Экологический мониторинг атмосферного воздуха,"{'Лидеры инноваций': 10.918853332200625, 'Меня..."
1,2,Перовскитные солнечные батареи для питания пор...,False,Перовскитные солнечные батареи для питания пор...,"{'Меняющие реальность': 45.82386533594941, 'Ли..."
2,3,Интерактивные прозрачные дисплеи,False,Интерактивные прозрачные дисплеи,"{'Меняющие реальность': 0.0, 'Лидеры инноваций..."
3,4,"Платформа для анализа логов, UX-мониторинга и ...",True,"Платформа для анализа логов, UX-мониторинга и ...","{'Лидеры инноваций': 4.981049792006358, 'Меняю..."


In [50]:
q = '''SELECT * FROM subsidies'''
df_sub = pd.read_sql(q, conn)
df_sub[df_sub['title'] == 'Меняющие реальность']

/var/folders/64/nbt92b3d4_n2r41g3gx6lm340000gn/T/ipykernel_33082/360825806.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sub = pd.read_sql(q, conn)


,id,count,title


In [53]:
df_sub

,id,count,title
0,1,0,Лидеры иноваций
1,2,0,Меняющие реальность


In [60]:
title_index = df_sub[df_sub['title'] == 'Меняющие реальность']['id'].iloc[0]

2

In [ ]:
class PredictController:
    def connect_db(self):
        self.conn = psycopg2.connect(dbname='ideas', user='ideas', 
                        password='ideas2022', host='collabro.ru')
        self.cursor = conn.cursor()

    def get_data_db(self):
        q = f'''SELECT subsidies.title AS subsidy, projects.title, subsidies.id,
                     subsidy_project.accepted FROM subsidy_project
                    LEFT JOIN subsidies ON (subsidy_project.subsidies_id = subsidies.id)
                    LEFT JOIN projects ON (subsidy_project.project_id = projects.id);'''
        df = pd.read_sql(q, self.conn)
        self.len_sub_project = len(df)

        df_train = df[df['accepted'] == True]

        df_test = pd.read_sql('SELECT * FROM projects', self.conn)
        return df_train, df_test
        
    def write_predictions(self, row):
        q = '''SELECT * FROM subsidies'''
        df_sub = pd.read_sql(q, self.conn)
        project_id = row['id']
        predictions = row['predictions']
        dsubsidy_project = pd.read_sql('SELECT * FROM subsidy_project', self.conn)
        for subsidy in predictions:
            subsidy_id = df_sub[df_sub['title'] == subsidy]['id'].iloc[0]
            prob = predictions[subsidy]
            print(prob, self.len_sub_project)
            if len(dsubsidy_project[(dsubsidy_project['project_id'] == project_id) & (dsubsidy_project['subsidies_id'] == subsidy_id)]) == 0:
                
                self.values.append((int(self.len_sub_project + 1), False, float(prob), int(project_id), int(subsidy_id)))
                self.len_sub_project += 1
            
            self.conn.commit()

    def fit_predict(self, categorys='subsidies', join_table='subsidy_project', category_col='title', category_id='subsidies_id'):
        self.categorys = categorys
        self.join_table=join_table
        self.category_col=category_col
        self.category_id=category_id
        self.connect_db()
        df_train, df_test = self.get_data_db()
        Predictor = CategoryPredictor(label='title',
                              category_columns=['subsidy'], df=df_train)
        Predictor.fit_all()
        df_predicted = Predictor.predict(labels=df_test, category='subsidy')
        self.values = []
        df_predicted.apply(self.write_predictions, axis=1)

        if self.values != []:
            q = '''INSERT INTO subsidy_project (id, accepted, probability, project_id, subsidies_id) VALUES {}'''
            records_list_template = ','.join(['%s'] * len(self.values))
            insert_query = q.format(records_list_template)
            cursor = self.conn.cursor()
            cursor.execute(insert_query, self.values)
            self.conn.commit()
        self.conn.close()
        return df_predicted

    def server(self):
        while True:
            self.fit_predict()
            time.sleep(10)